# Cassava Leaf Disease Classification - Exploratory Data Analysis / キャッサバの葉の病気分類 - 探索的データ分析

### Japanese translation of [Cassava Leaf Disease Classification - Exploratory Data Analysis](https://www.kaggle.com/ihelon/cassava-leaf-disease-exploratory-data-analysis)

Quick Exploratory Data Analysis for [Cassava Leaf Disease Classification](https://www.kaggle.com/c/cassava-leaf-disease-classification) challenge   
[キャッサバの葉の病気分類](https://www.kaggle.com/c/cassava-leaf-disease-classification)チャレンジのための簡単な探索的データ分析   


This competition will challenge you to distinguish between several diseases that cause material harm to the food supply of many African countries. In some cases the main remedy is to burn the infected plants to prevent further spread, which can make a rapid automated turnaround quite useful to the farmers.   
このコンペティションでは、多くのアフリカ諸国の食糧供給に重大な害を及ぼすいくつかの病気を区別することが求められます。病気の対策として感染した植物を燃やすことで拡大を防ぐことが行われることもあり、これによる迅速な回復は農家にとって非常に有益です。

![](https://storage.googleapis.com/kaggle-competitions/kaggle/13836/logos/header.png)

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:black; background:#5BEB9C; border:0' role="tab" aria-controls="home"><center>Quick Navigation / クイックナビゲーション</center></h3>

* [1. Overview / 概要](#1)
* [2. General Visualization / 一般的な可視化](#2)
* [3. Cassava Bacterial Blight (CBB) / キャッサバ・バクテリア・ブライト (CBB)](#3)
* [4. Cassava Brown Streak Disease (CBSD) / キャッサバ褐色条斑病 (CBSD)](#4)
* [5. Cassava Green Mottle (CGM) / キャッサバ・グリーン・モットル (CGM)](#5)
* [6. Cassava Mosaic Disease (CMD) / キャッサバモザイク病 (CMD)](#6)
* [7. Healthy / 健康](#7)
    
    
* [Augmentation Examples / データ拡張の例](#50)
    
    
* [Submission Example / 提出ファイルの例](#100)

<a id="1"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>Overview / 概要<center><h2>

In [ ]:
import os
import json

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import cv2
import albumentations as A
from sklearn import metrics as sk_metrics

In [ ]:
BASE_DIR = "../input/cassava-leaf-disease-classification/"

In this competition we have 5 classes: **4 diseases** and **1 healthy**   
We can find the mapping between the class number and its name in the file label_num_to_disease_map.json   
このコンペティションでは5つの分類を行います。: **4つの病気と1つの健康**   
クラス番号とその名前のマッピングは、label_num_to_disease_map.json ファイルにあります。

In [ ]:
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    
print(json.dumps(map_classes, indent=4))

In [ ]:
input_files = os.listdir(os.path.join(BASE_DIR, "train_images"))
print(f"Number of train images: {len(input_files)}")

Let's take a look at the dimensions of the first 300 images   
As you can see below, all images are the same size (600, 800, 3)   
最初の300枚の画像の寸法を見てみましょう。   
以下に示すように全ての画像は同じサイズ（600、800、3）です。

In [ ]:
img_shapes = {}
for image_name in os.listdir(os.path.join(BASE_DIR, "train_images"))[:300]:
    image = cv2.imread(os.path.join(BASE_DIR, "train_images", image_name))
    img_shapes[image.shape] = img_shapes.get(image.shape, 0) + 1

print(img_shapes)

Let's load the training dataframe and add a column with the real class name to it.   
トレーニングデータのデータフレームをロードし、実際のクラス名を含むカラムを追加しましょう。

In [ ]:
df_train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

df_train["class_name"] = df_train["label"].astype(str).map(map_classes)

df_train

Let's look at the number of pictures in each class.   
各クラスの写真の数を見てみましょう。

In [ ]:
plt.figure(figsize=(8, 4))
sn.countplot(x="label", data=df_train);

As we can see, the dataset has a fairly large imbalance.   
このようにデータセットにはかなり大きな不均衡があります。

<a id="2"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>General Visualization / 一般的な可視化<center><h2>

In [ ]:
def visualize_batch(image_ids, labels):
    plt.figure(figsize=(16, 12))
    
    for ind, (image_id, label) in enumerate(zip(image_ids, labels)):
        plt.subplot(3, 3, ind + 1)
        image = cv2.imread(os.path.join(BASE_DIR, "train_images", image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(f"Class: {label}", fontsize=12)
        plt.axis("off")
    
    plt.show()

In [ ]:
tmp_df = df_train.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["class_name"].values

visualize_batch(image_ids, labels)

<a id="3"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>Cassava Bacterial Blight (CBB) / キャッサバ・バクテリア・ブライト (CBB)<center><h2>

<img style="height:300px" src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1865449%2Fbe9cdd94efb9b1660066ad10b55c8626%2Fbact_bright.jpeg?generation=1605827469211692&alt=media">
<cite>The image from discussion / ディスカッションからの写真: <a href="https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198143">Cassava Leaf Diseases: Overview / キャッサバの葉の病気: 概要</a></cite>

In [ ]:
tmp_df = df_train[df_train["label"] == 0]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

<a id="4"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>Cassava Brown Streak Disease (CBSD) / キャッサバ褐色条斑病 (CBSD)<center><h2>

<img style="height:300px" src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1865449%2Ffeba3dafc914d04517659650d137b77a%2Fbrown_st.jpeg?generation=1605830407530983&alt=media">
<cite>The image from discussion / ディスカッションからの写真: <a href="https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198143">Cassava Leaf Diseases: Overview / キャッサバの葉の病気: 概要</a></cite>

In [ ]:
tmp_df = df_train[df_train["label"] == 1]
print(f"Total train images for class 1: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

<a id="5"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>Cassava Green Mottle (CGM) / キャッサバ・グリーン・モットル (CGM)<center><h2>

<img style="height:300px" src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1865449%2F4f2975866feb2a1d4ef4111c2d57db29%2Fgreen_mottle.jpeg?generation=1605829101431013&alt=media">
<cite>The image from discussion / ディスカッションからの写真: <a href="https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198143">Cassava Leaf Diseases: Overview / キャッサバの葉の病気: 概要</a></cite>

In [ ]:
tmp_df = df_train[df_train["label"] == 2]
print(f"Total train images for class 2: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

<a id="6"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>Cassava Mosaic Disease (CMD) / キャッサバモザイク病 (CMD)<center><h2>

<img style="height:300px" src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1865449%2F36990f77ded6667e5c30d19b5405d4d3%2Fmosaic_disease.jpeg?generation=1605829705010773&alt=media">
<cite>The image from discussion / ディスカッションからの写真: <a href="https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198143">Cassava Leaf Diseases: Overview / キャッサバの葉の病気: 概要</a></cite>

In [ ]:
tmp_df = df_train[df_train["label"] == 3]
print(f"Total train images for class 3: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

<a id="6"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>Healthy / 健康<center><h2>

In [ ]:
tmp_df = df_train[df_train["label"] == 4]
print(f"Total train images for class 4: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

<a id="50"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>Augmentation Examples / データ拡張の例<center><h2>

In [ ]:
def plot_augmentation(image_id, transform):
    plt.figure(figsize=(16, 4))
    img = cv2.imread(os.path.join(BASE_DIR, "train_images", image_id))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")

    plt.subplot(1, 3, 3)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")
    
    plt.show()

Since we have a fairly limited number of some classes, we can use augmentation   
This section shows examples of augmentation using the [albumentations](https://albumentations.ai/) library   
一部のクラスの数はかなり限られているため、データ拡張を使用します。   
ここでは、 [albumentations](https://albumentations.ai/) ライブラリを使用したデータ拡張の例を示します。

The example below uses rotate-shift-scale augmentation with specular edge complementation. For this kind of pictures, this augmentation looks quite natural.   
以下の例では、鏡面エッジ補完を使用した回転・シフト・スケール拡張を使用しています。この種の写真の場合、このデータ拡張は非常に自然に見えます。

In [ ]:
transform = A.ShiftScaleRotate(
    p=1.0, 
    shift_limit=(-0.3, 0.3), 
    scale_limit=(-0.1, 0.1), 
    rotate_limit=(-180, 180), 
    interpolation=0, 
    border_mode=4, 
)

plot_augmentation("1003442061.jpg", transform)

Another useful augmentation could be CoarseDropout. Thanks to this augmentation, you can complicate the life of the model so that she does not look too closely at some of the details of the image.   
Let's look at the example below:   
もう1つの便利な拡張機能はCoarseDropoutです。この拡張機能により、モデルを複雑にして、モデルが画像の細部の一部をあまり詳しく見ないようにすることができます。   
以下の例を見てみましょう。

In [ ]:
transform = A.CoarseDropout(
    p=1.0, 
    max_holes=100, 
    max_height=50, 
    max_width=50, 
    min_holes=30, 
    min_height=20, 
    min_width=20,
)

plot_augmentation("1003442061.jpg", transform)

<a id="100"></a>
<h2 style='background:#5BEB9C; border:0; color:black'><center>Submission Example / 提出ファイルの例<center><h2>

Load the submission template   
提出ファイルのテンプレートをロードします。

In [ ]:
df_sub = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv", index_col=0)
df_sub

As we can see only one file in the submission file   
提出ファイルには1つのファイルしかありません。

In [ ]:
os.listdir(os.path.join(BASE_DIR, "test_images"))

This is because [it is a Code Competition](https://www.kaggle.com/c/cassava-leaf-disease-classification/overview/code-requirements), and the test data is hidden   
Your notebook should correct working with unseen test dataset   
これは、[コード・コンペティション](https://www.kaggle.com/c/cassava-leaf-disease-classification/overview/code-requirements)であるため、テストデータが非表示になっています。   
ノートブックは、目に見えないテストデータセットでの修正作業を行う必要があります。

Let's choose the most popular class of training set as the label for all images in test set   
テストデータセット内のすべての画像のラベルの中で最も数の多いクラスのデータを選択しましょう。

In [ ]:
df_sub["label"] = 3

For example, let's calculate our accuracy (competition metric) with this forecast for a train data   
たとえば、トレインデータのこの予測を使用して、精度（コンペティション指標）を計算してみましょう。

In [ ]:
y_true = df_train["label"].values
y_pred = np.full_like(y_true, 3)
print(f"accuracy score (predict 3): {sk_metrics.accuracy_score(y_true, y_pred):.3f}")

in the same way, you can calculate the estimated accuracy for other predictions:   
同様に、他の予測についても精度を計算できます。

In [ ]:
for pred_class in range(0, 5):
    y_true = df_train["label"].values
    y_pred = np.full_like(y_true, pred_class)
    print(f"accuracy score (predict {pred_class}): {sk_metrics.accuracy_score(y_true, y_pred):.3f}")

Write result to the submission file   
結果を提出ファイルに書き込む。

In [ ]:
df_sub.to_csv("submission.csv")

If you submit the result for evaluation, you will get an accuracy of 0.614 on a public liderboard (on the train it is 0.615). This may indicate that there is also an imbalance of classes on the public test distribution.   
評価のために結果を送信すると、パブリックリーダーボードで0.614の精度が得られます（トレインデータでは0.615です）。 これは、パブリックテスト配布のクラスにも不均衡があることを示している可能性があります。